In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import pyodbc
import nest_asyncio
import os 
import calendar
from datetime import datetime
from tqdm import tqdm 

nest_asyncio.apply()
pd.options.mode.chained_assignment = None # suppressed warning if writing to df


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [5]:
today = datetime.today()
current_year = today.year

In [6]:
import calendar, datetime as dt

def get_date(day, month, year):
    """
    Takes three integer values: day, month and year, and converts them to a datetime object
    """
    # handle two digit years 
    if year < 100:
        year += 1900 if year >= 50 else 2000
        
    calendar_days = calendar.monthrange(year, month)[1]
    # check if day is less than 1 and adjust to previous month
    if day < 1:
        # shift back month by 1
        
        month -= 1 
        if month < 1:
            # adjust the month 
            month = 12 
            year -= 1
        
        calendar_days = calendar.monthrange(year, month)[1]
        day = calendar_days

    if day > calendar_days:
        date_obj = datetime(year, month, 1) + dt.timedelta(day - 1)
    else:
        date_obj = datetime(year, month, day)

      
    return date_obj.date()

In [7]:
# list of frequently used columns
cols = ['AGE',
 'BLDSTOOL',
 'CHECKUP',
 'COLNCNCR',
 'COLNSCPY',
 'COLNTEST',
 'FINALWT',
 'HADMAM',
 'HADSGCOL',
 'HADSIGM',
 'HIVRISK',
 'HIVTEST',
 'HLTHPLAN',
 'HOWLONG',
 'IDAY',
 'IMONTH',
 'INCOME2',
 'IYEAR',
 'LASTSIGM',
 'LSTBLDST',
 'PERSDOC',
 'SDNATEST',
 'SEX',
 'SIGMSCPY',
 'STOOLDNA',
 'VCLTEST',
 'VIRCOLON',
 '_AGEG5YR']

# column maps 
colmap = {
    'age': 'AGE',
 '_ageg5yr': '_AGEG5YR',
 'persdoc': 'PERSDOC',
 'checkup': 'CHECKUP',
 'sex': 'SEX',
  'hlthplan': 'HLTHPLAN',
 'hivtest': 'HIVTEST',
 'hivrisk': 'HIVRISK',
 'wcol': 'FINALWT',
 'income2': 'INCOME2',
 'howlong': 'HOWLONG',
 'hadmam': 'HADMAM',
 'hadsigm': 'HADSIGM',
 'hadsgcol': 'HADSGCOL',
 'bldstool': 'BLDSTOOL',
 'lastsigm': 'LASTSIGM',
 'lstbldst': 'LSTBLDST',
 'colntest': 'COLNTEST',
 'stooldna': 'STOOLDNA',
 'vclntest': 'VCLNTEST',
 'sdnatest': 'SDNATEST',
 'vircolon': 'VIRCOLON',
 'colnscpy': 'COLNSCPY',
 'sigmscpy': 'SIGMSCPY',
 'colncncr': 'COLNCNCR',
 'iyear': 'IYEAR',
 'imonth': 'IMONTH',
 'iday': 'IDAY'
}

# update to start year at the most recent year of update to prevent attempting to build this dataset from scratch
start_year = 2022
year = 2022

In [9]:
# Read the BRFSS data breakdown file which contains the updated column names
df = pd.read_excel(r"C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\Input folder - zip\BRFSS_data_breakdown.xlsx", "vars", index_col='year')
# dictionary to host DataFrames 
frames = {}

In [10]:
# check if dataset already exists 
# accumulated_path = r'S:\LON_SSRES\DataAnalytics\US Life Science\Data\Input\BRFSS_COLORECTAL.csv'
accumulated_path = r'C:\Users\AnkitB\OneDrive - crisil.com\backup\Bernstein\Input folder - zip\BRFSS Filtered Dataset - Copy.csv'

if os.path.exists(accumulated_path):
    # if dataset exists, then we want to import the data so that it can be updated
    df2 = pd.read_csv(accumulated_path, parse_dates=['IDATE'])
    df2.rename(columns = {'VCLTEST': 'VCLNTEST'}, inplace=True)
else:
    # if the dataset doesn't exist, create an empty dataframe. Reduces chance of error
    df2 = pd.DataFrame()

In [11]:
# duplicate df2 then append the new dat to the frame
data = pd.DataFrame()

In [12]:
# Create progress bar to track progres of the data reading
bar = tqdm(df.loc[start_year:, :].iterrows(), desc=('Reading %s row:' % year))
# bar = tqdm(df.iterrows(), desc=('Reading %s row:' % year))

for year, row in bar:
    # identify file location and create output path name in case of code break 
    folder, filename = os.path.split(row.fileloc)
    output_filename = os.path.splitext(filename)[0] +'.csv'

    bar.set_description('Reading %s data' % year)
    df1 = pd.read_sas(row.fileloc)
    row = row.dropna()
    indx = [x for x in row.index if not x in ['fileloc']]
 
    
    bar.set_description(f'Updating {year} column names...')
    df1 = df1.rename(columns={v.upper():colmap[k] for k,v in row.loc[indx].items()})
    df1['IDATE'] = df1.apply(lambda row: get_date(day=int(row.IDAY), month=int(row.IMONTH), year=int(row.IYEAR)),axis=1)
    df1['DATAYEAR'] = year

    bar.set_description(f'Appending {year} to full dataset ...')
    data = pd.concat([data, df1], ignore_index=True)

    bar.set_description(f'Saving {year} as {filename}.csv ...')
    df1.to_csv(os.path.join(folder, output_filename), index=False)


    bar.set_description(f'Adding {year} to frames dictionary ...')
    frames[year] = df1 



Reading 2022 data: : 0it [00:00, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'R:\\Life science tools and diagnostics\\Topical analyses\\EXAS deep dive\\BRFSS\\Jasmeen\\Data\\LLCP2022XPT\\LLCP2022.XPT'

In [ ]:
# for year, row in df.loc[1998:2010].iterrows():
#     folder, filename = os.path.split(row.fileloc)
#     output_filename = os.path.splitext(filename)[0] +'.csv'
#     df1 = pd.read_csv(os.path.join(folder,output_filename))
#     types = df1.dtypes
#     notfloats = types[~types.eq(float)]
#     for col in notfloats.keys():
#         try:
#             df1[col] = df1[col].apply(lambda x: eval(x).decode('utf-8'))
#         except:
#             continue
#         if col == 'INTVID':
#             continue
#         elif col == 'IDATE':
#             df1[col] = df1[col].apply(get_date)
#         else:
#             try:
#                 df1[col] = df1[col].apply(lambda x: float(x) if len(x) > 0 else None)
#             except:
#                 continue

#     # update float types again:
#     types = df1.dtypes
#     floats_only = types[types.eq(float)]
#     for col in floats_only.keys():
#         df1[col] = df1[col].apply(lambda x: x if not x.is_integer() else int(x))
        
#     # rename columns
#     row = row.dropna()
#     indx = [x for x in row.index if not x in ['fileloc']]
    
#     df1 = df1.rename(columns={v:colmap[k] for k,v in row.loc[indx].items()})
#     print('Updating', output_filename, '...')
#     df1.to_csv(os.path.join(folder,output_filename), index=False)

C:\Users\jtatan\AppData\Local\Temp\ipykernel_26204\3616385677.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(os.path.join(folder,output_filename))


Updating CDBRFS98.csv ...
Updating CDBRFS99.csv ...
Updating CDBRFS00.csv ...
Updating CDBRFS01.csv ...
Updating cdbrfs02.csv ...
Updating cdbrfs03.csv ...
Updating CDBRFS04.csv ...
Updating CDBRFS05.csv ...
Updating CDBRFS06.csv ...
Updating CDBRFS07.csv ...
Updating CDBRFS08.csv ...
Updating CDBRFS09.csv ...
Updating CDBRFS10.csv ...


0it [00:00, ?it/s]

In [164]:
# from tqdm import tqdm 
# bar = tqdm(df.loc[start_year:].iterrows())
# # bar = tqdm(frames.items())
# # this will hold the data from the newly added data
# data = pd.DataFrame()

# for year, row in bar:
#     # get the file location
#     folder, filename = os.path.split(row.fileloc)
#     # define output file hname
#     output_filename = os.path.splitext(filename)[0] +'.csv'
    

#     bar.set_description(f'Reading {output_filename}...')
    
#     if not year in frames:
#         # read the CSV file that was created earlier to 
#         df1 = pd.read_csv(os.path.join(folder,output_filename), usecols=lambda x: x in cols,dtype={'IMONTH': bytes, 'IDAY': bytes, 'IYEAR':bytes})
#         # the day, month and year variables might be stored as bytes. This will transform them into ints 
#         df1['IDAY'] = df1['IDAY'].apply(eval).astype(int)
#         df1['IMONTH'] = df1['IMONTH'].apply(eval).astype(int)
#         df1['IYEAR'] = df1['IYEAR'].apply(eval).astype(int)

#     else:
#         df1 = frames[year]

#     df1['IDATE'] = df1.apply(lambda row: get_date(day=int(row.IDAY), month=int(row.IMONTH), year=int(row.IYEAR)),axis=1)
#     df1['DATAYEAR'] = year
#     data = pd.concat([data, df1])

Reading LLCP2023.csv...: : 3it [03:40, 73.47s/it]


In [165]:
data1 = pd.concat([df2, data])

In [167]:
data1.to_csv(r'S:\LON_SSRES\DataAnalytics\US Life Science\Data\Input\2024\BRFSS_COLORECTAL.csv',index=False)

In [168]:
types = {'int64': 'numeric(18,0)', 'float64': 'decimal(28,0)', 'object': 'varchar(50)', 'datetime64[ns]': 'datetime' }
dtypes = data.dtypes

In [170]:
# print(',\n'.join(f'[{k}] {types[v.name]}' for k, v in dtypes.items()))

KeyError: 'int32'

: 